In [1]:
import importlib
from pathlib import Path

import pandas as pd
from rich.console import Console

c = Console(width=60)

GDRIVE = Path("~/Google Drive/Shared drives/Solutions").expanduser()

# Quick Start

In [2]:
from google.genai.types import ThinkingConfig, ThinkingLevel

from cuery import Prompt, Response, Task, set_env

set_env(apify_secrets=False)


# Define response model
class Entity(Response):
    name: str
    type: str


class NamedEntities(Response):
    entities: list[Entity]


context = pd.DataFrame(
    {
        "text": [
            "Apple is headquartered in Cupertino, California.",
            "Barack Obama was the 44th President of the United States.",
            "The Eiffel Tower is located in Paris, France.",
        ]
    }
)

prompt = Prompt.from_string("Extract named entities from the following text: {{text}}")
task = Task(prompt=prompt, response=NamedEntities)

result = await task(
    context,
    model="google/gemini-3-flash-preview",
    n_concurrent=10,
    thinking_config=ThinkingConfig(thinking_level=ThinkingLevel.LOW),
)
print(result.to_pandas(explode=False))

Gathering searches:   0%|          | 0/3 [00:00<?, ?it/s]

                                                text  \
0   Apple is headquartered in Cupertino, California.   
1  Barack Obama was the 44th President of the Uni...   
2      The Eiffel Tower is located in Paris, France.   

                    entities_name                       entities_type  
0  [Apple, Cupertino, California]  [ORGANIZATION, LOCATION, LOCATION]  
1   [Barack Obama, United States]                  [PERSON, LOCATION]  
2   [Eiffel Tower, Paris, France]           [LOCATION, CITY, COUNTRY]  


In [3]:
task.queries[0]

{'model': 'gemini-3-flash-preview',
 'config': GenerateContentConfig(
   safety_settings=[
     SafetySetting(
       category=<HarmCategory.HARM_CATEGORY_HARASSMENT: 'HARM_CATEGORY_HARASSMENT'>,
       threshold=<HarmBlockThreshold.OFF: 'OFF'>
     ),
     SafetySetting(
       category=<HarmCategory.HARM_CATEGORY_HATE_SPEECH: 'HARM_CATEGORY_HATE_SPEECH'>,
       threshold=<HarmBlockThreshold.OFF: 'OFF'>
     ),
     SafetySetting(
       category=<HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: 'HARM_CATEGORY_SEXUALLY_EXPLICIT'>,
       threshold=<HarmBlockThreshold.OFF: 'OFF'>
     ),
     SafetySetting(
       category=<HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: 'HARM_CATEGORY_DANGEROUS_CONTENT'>,
       threshold=<HarmBlockThreshold.OFF: 'OFF'>
     ),
     SafetySetting(
       category=<HarmCategory.HARM_CATEGORY_CIVIC_INTEGRITY: 'HARM_CATEGORY_CIVIC_INTEGRITY'>,
       threshold=<HarmBlockThreshold.OFF: 'OFF'>
     ),
   ],
   system_instruction='',
   thinking_config=Thinking

In [4]:
result[0]._raw_response

GenerateContentResponse(
  candidates=[
    Candidate(
      content=Content(
        parts=[
          Part(
            function_call=FunctionCall(
              args=<... Max depth ...>,
              name=<... Max depth ...>
            ),
            thought_signature=b'\x124\n2\x01r\xc8\xda|\xe7\xc4\x0f\xda\xefG\xa3}\xeb\x12\x03Q\x83\xe1X\x7f\x82\xb7!K\x16\xa99\x9e\xa1\xd3\x84\xba~\xf1}\x14\x1b\xb7t\xf2\xd3\x96\x82X\x8e\xf9*\xa6\xb6'
          ),
        ],
        role='model'
      ),
      finish_reason=<FinishReason.STOP: 'STOP'>,
      index=0
    ),
  ],
  model_version='gemini-3-flash-preview',
  response_id='GnVFae7hMMm3vdIP2v7n-QU',
  sdk_http_response=HttpResponse(
    headers=<dict len=11>
  ),
  usage_metadata=GenerateContentResponseUsageMetadata(
    candidates_token_count=50,
    prompt_token_count=85,
    prompt_tokens_details=[
      ModalityTokenCount(
        modality=<MediaModality.TEXT: 'TEXT'>,
        token_count=85
      ),
    ],
    total_token_count=135


In [9]:
print(result.to_pandas(explode=True))

                                                text           name  \
0   Apple is headquartered in Cupertino, California.          Apple   
1   Apple is headquartered in Cupertino, California.      Cupertino   
2   Apple is headquartered in Cupertino, California.     California   
3  Barack Obama was the 44th President of the Uni...   Barack Obama   
4  Barack Obama was the 44th President of the Uni...  United States   
5      The Eiffel Tower is located in Paris, France.   Eiffel Tower   
6      The Eiffel Tower is located in Paris, France.          Paris   
7      The Eiffel Tower is located in Paris, France.         France   

           type  
0  Organization  
1      Location  
2      Location  
3        Person  
4      Location  
5      LOCATION  
6      LOCATION  
7      LOCATION  


# Prompts

In [ ]:
from cuery import Prompt

prompt = Prompt.from_string("Explain {{ topic }} in simple terms.")
c.print(prompt)

# Context

In [ ]:
from cuery.context import iter_context

df = pd.DataFrame(
    {
        "topic": ["Machine Learning", "Natural Language Processing", "Computer Vision"],
        "audience": ["beginners", "developers", "researchers"],
    }
)

contexts, count = iter_context(df, required=["topic", "audience"])
next(contexts)

# Response, Task

In [ ]:
from cuery import Field, Prompt, Response, Task


class MovieRecommendation(Response):
    title: str
    year: int = Field(gt=1900, lt=2030)
    genre: list[str]
    rating: float = Field(ge=0, le=10)


# Multi-output response model
class MovieRecommendations(Response):
    recommendations: list[MovieRecommendation]


movie_prompt = Prompt.from_string(
    "Recommend a movie for {{ audience }} interested in {{ topic }}."
)
context = [
    {"topic": "Machine Learning", "audience": "beginners"},
    {"topic": "Computer Vision", "audience": "researchers"},
]

movie_task = Task(prompt=movie_prompt, response=MovieRecommendations)
result = await movie_task(context)
print(result)
print(result.to_pandas())

In [ ]:
print(result.usage())

In [ ]:
from cuery import pprint

print(result[0]._raw_response.model)
print(result[0]._raw_response.service_tier)

In [ ]:
print(movie_task.queries[0]["messages"][0]["content"])

In [ ]:
c.print(result[0])

# Chain

In [ ]:
from typing import Literal

from cuery import Chain, Task

# Reuse example from above
movies = Task(prompt=movie_prompt, response=MovieRecommendations)


# Add PG rating
class Rating(Response):
    pg_category: Literal["G", "PG", "PG-13", "R", "NC-17"]
    """PG rating of the movie."""
    pg_reason: str
    """Reason for the rating."""


rating_prompt = Prompt.from_string("What is the PG rating for {{ title }}?")
rating_task = Task(prompt=rating_prompt, response=Rating)

# Create a chain of tasks
chain = Chain(movie_task, rating_task)
result = await chain(context, model="openai/gpt-3.5-turbo", n_concurrent=20)
result

In [ ]:
print(result)

# Templates

In [ ]:
from cuery import templates
from cuery.utils import render_template

prompt = """
Analyze the following record and provide a summary in Markdown format.
Make sure to include the name, age, city, and interests.
"""

context = {
    "record": {
        "name": "John Doe",
        "age": 30,
        "city": "New York",
        "interests": ["reading", "traveling", "coding"],
    }
}


In [ ]:
for mode in ["json", "text", "md"]:
    tpl = templates.prompt_with_template(prompt, template=f"record_to_{mode}")
    print(render_template(tpl, **context), end="\n\n")